# AlphaFold2-Multimer Inference Endpoints

AlphaFold2-Multimer NIM provides the following endpoints:

- `protein-structure/alphafold2/multimer/predict-structure-from-sequences` - Predict a protein structure given an input list of amino acide sequences.

- `protein-structure/alphafold2/multimer/predict-MSA-from-sequences` - Perform a Multiple Sequence Alignment (MSA) and return the MSA and templates for AlphaFold2 inference. This endpoint is useful for batching long-running and CPU-intensive MSA runs prior to structure prediction.

- `protein-structure/alphafold2/multimer/predict-structure-from-MSA` - Perform structural prediction from an input MSA and templates. This is useful when using a pre-computed or custom/external MSA.

**Usage**
Below, we outline the three endpoints of the API. We give real examples of requests that should run when the NIM is correctly configured.

## Predict Structure from Multiple Input Sequences (Multimers)

The `predict-structure-from-sequences` endpoint provides a full end-to-end structural prediction pipeline, i.e. from protein sequences to a multimer protein structure. It requires at least 1 and up to 6 amino acid sequences, though there are many tunable parameters:

- `sequences`: An array of valid amino acid sequences. Refer to the table of **[amino acid codes](https://www.ebi.ac.uk/pdbe/docs/roadshow_tutorial/msdtarget/AAcodes.html)** if you are unsure if your sequence is valid.

- `databases`: A list containing any of `uniref90`, `mgnify`, and `small_bfd`. These databases contain sequences used to generate a Multiple Sequence Alignment (MSA) that is used as input to the structural prediction neural network in AlphaFold2. In general, passing all three databases will provide the most accurate structural prediction at the cost of requiring the longest runtime.

- `algorithm`: The algorithm used for Multiple Sequence Alignment. Currently, only `jackhmmer` is supported.

- `e_value`: The sequence e-value for filtering sequences in the MSA. Smaller implies stricter alignments - fewer sequences with higher probability of origin will be included, but this will also reduce the sensitivity of the MSA. The default value of `0.0001` is in general a good choice. This value ranges from 0 to 1.

- `bit_score`: The sequence bit-score to use for filtering before MSA. If passed, this is used in place of e-value for filtering. A good starting place is around `200`. This value is greater than zero.

- `iterations`: The number of MSA iterations to perform. In general, the default `iterations=1` is sufficient and takes the least amount of time.

- `relax_prediction`: Set to `True` to run structural relaxation after prediction. This is set to `True` by default and helps fix clashes in the predicted structure.

In [ ]:
import requests
import json

url = "http://localhost:8000/protein-structure/alphafold2/multimer/predict-structure-from-sequences"
sequences = ["MNVIDIAIAMAI", "IAMNVIDIAAI"]  # Replace with the actual sequences you want to perform structure prediction on.

headers = {
    "content-type": "application/json"
}

data = {
    "sequences": sequences,
    "databases": ["uniref90", "mgnify", "small_bfd"]
}

response = requests.post(url, headers=headers, data=json.dumps(data))

# Check if the request was successful
if response.ok:
    print("Request succeeded:", response.json())
else:
    print("Request failed:", response.status_code, response.text)

The output of this endpoint is a PDB file. The PDB format can easily be viewed using [pymol](https://www.pymol.org/) and other viewing programs; see the [pymol](https://www.pymol.org/) website for documentation and usage.

## Predict MSA from Multiple Input Sequences (Multimers)

The `predict-msa-from-sequences` endpoint generates Multiple Sequence Alignments (MSAs) and templates used for structural prediction. This is useful if you want to batch prediction on different (CPU-intensive) nodes.

In [ ]:
import requests
import json

url = "http://0:8000/protein-structure/alphafold2/multimer/predict-msa-from-sequences"
sequences = ["MNVIDIAIAMAI", "IAMNVIDIAAI"]  # Replace with the actual sequences you want to perform structure prediction on.

headers = {
    "content-type": "application/json"
}

data = {
    "sequences": sequences,
    "databases": ["uniref90", "mgnify", "small_bfd"]
}

response = requests.post(url, headers=headers, data=json.dumps(data))

# Check if the request was successful
if response.ok:
    print("Request succeeded:", response.json())
else:
    print("Request failed:", response.status_code, response.text)

The `predict-msa-from-sequences` endpoint takes the following parameters:

- `sequences`: An array of valid amino acid sequences. Refer to the table of **[amino acid codes](https://www.ebi.ac.uk/pdbe/docs/roadshow_tutorial/msdtarget/AAcodes.html)** if you are unsure if your sequence is valid.

- `databases`: A list containing any of `uniref90`, `mgnify`, and `small_bfd`. These databases contain sequences used to generate a Multiple Sequence Alignment (MSA) that is used as input to the structural prediction neural network in AlphaFold2. In general, passing all three databases will provide the most accurate structural prediction at the cost of requiring the longest runtime. If you must pick only one, `uniref90` is considered the best choice, though it is still recommended to run with all three.

- `algorithm`: The algorithm used for Multiple Sequence Alignment. Currently, only `jackhmmer` is supported.

- `e_value`: The sequence e-value for filtering sequences in the MSA. Smaller implies stricter alignments - fewer sequences with higher probability of origin will be included, but this will also reduce the sensitivity of the MSA. The default value of `0.0001` is in general a good choice. This value ranges from 0 to 1.

- `bit_score` The sequence bit-score to use for filtering before MSA. If passed, this is used in place of e-value for filtering. A good starting place is around `200`. This value is greater than zero.

- `iterations`: The number of MSA iterations to perform. In general, the default `iterations=1` is sufficient and takes the least amount of time.

## Predict Protein Structure from MSAs

The `predict-structure-from-msa` endpoint takes the results of the predict-msa-from-sequences endpoint and runs structural prediction.

**Note**: We do not recommend running the msa-to-structure prediction using CURL. This is because the inputs have characters that require careful escaping in bash. For the best user experience, we recommend interacting with this endpoint via the Python requests module.

The `predict-structure-from-msa` endpoint takes the following arguments:

- `sequences`: An array of valid amino acid sequences. Refer to the table of **[amino acid codes](https://www.ebi.ac.uk/pdbe/docs/roadshow_tutorial/msdtarget/AAcodes.html)** if you are unsure if your sequence is valid.

- `alignments`: The MSA results from `predict-msa-from-sequences`. This is an array of dictionaries of tuples of the form: `{&lt;db name&gt; : {&lt;db name&gt;, &lt;MSA output&gt;, &lt;MSA output format&gt;}}`, one for each input amino acid sequence.

- `templates`: Templates from the structural database search. These are in a format specific to the internals of AlphaFold2; more detils of the fields can be found here.

- relax_prediction: Set to True to run structural relaxation after prediction. This is set to True by default and helps fix clashes in the predicted structure.